In [1]:
import pybedtools

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import seaborn.objects as so

from scipy.stats import pearsonr, spearmanr
import glob

In [34]:
def flatten_tissues_expand_df(tfbs_df):
    tfbs_part_repeat = tfbs_df.iloc[:,:9]
    tfbs_part_flatten = tfbs_df.iloc[:,9:]
    
    repeat_df =  pd.DataFrame(np.repeat(tfbs_part_repeat, len(tfbs_part_flatten.columns), axis=0))
    flatten_arr = tfbs_part_flatten.to_numpy().flatten()

    tissue_repeat = np.tile(np.array(tfbs_part_flatten.columns), len(tfbs_part_repeat))
    
    repeat_df.columns = tfbs_part_repeat.columns
    repeat_df["All_tissues"] = flatten_arr
    repeat_df["Tissue"] = tissue_repeat
    

    return repeat_df

In [5]:
path = "/sybig/projects/GeneRegulation/data/jme/Bachelorarbeit/data/Single_TFBS/All_TFBS.csv"
df_combined = pd.read_csv(path)

In [35]:
flat_df = flatten_tissues_expand_df(df_combined)

In [36]:
flat_df

,chr,geneID,tf,close_tss,dist_tss,strand_orientation,homotypic_count,all_tfbs_count,all_tfbs_unique_count,All_tissues,Tissue
0,chr1,ENSG00000163209.14,HMBOX1,159,169,nT,1,24,20,0.691,Adipose - Subcutaneous
1,chr1,ENSG00000163209.14,HMBOX1,159,169,nT,1,24,20,1.043,Adipose - Visceral (Omentum)
2,chr1,ENSG00000163209.14,HMBOX1,159,169,nT,1,24,20,1.906,Adrenal Gland
3,chr1,ENSG00000163209.14,HMBOX1,159,169,nT,1,24,20,1.397,Artery - Aorta
4,chr1,ENSG00000163209.14,HMBOX1,159,169,nT,1,24,20,1.385,Artery - Coronary
...,...,...,...,...,...,...,...,...,...,...,...
37849459,chr7,ENSG00000128512.20,STAT5B,105,120,T,1,104,79,2.226,Testis
37849460,chr7,ENSG00000128512.20,STAT5B,105,120,T,1,104,79,7.504,Thyroid
37849461,chr7,ENSG00000128512.20,STAT5B,105,120,T,1,104,79,11.100,Uterus
37849462,chr7,ENSG00000128512.20,STAT5B,105,120,T,1,104,79,6.597,Vagina


In [38]:
sns.set_theme(style="ticks")
fig, ax = plt.subplots()
#ax.set_xscale("log", base=2)

sns.relplot(flat_df, x="close_tss", y="All_tissues", hue="Tissue")


ax.xaxis.grid(True)
ax.set(ylabel="")
ax.set_ylabel("Gene Expression")
#fig.savefig("/sybig/projects/GeneRegulation/data/jme/Bachelorarbeit/data/Single_TFBS/Genexpression_All_TFBS.png")
plt.show()